In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# stock_prices = pd.read_csv(os.path.join(dirname, "stock_prices.csv"))

train = pd.read_csv("train.csv")



In [5]:
train[["matched_size", "imbalance_size", "imbalance_buy_sell_flag"]]

train[train.matched_size == 0]

train["matched_ratio"] = train.imbalance_buy_sell_flag*train.imbalance_size/train.matched_size
# train["test_ratio"] = train.imbalance_buy_sell_flag*train.matched_size/train.imbalance_size

In [6]:
stock_id = train[train.matched_ratio.isna()].stock_id.unique()
train[train.stock_id.isin(stock_id)].seconds_in_bucket.unique()

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540])

here we can see that some stocks don't have an auction

In [7]:

def make_data(df, train=0, full_data=False):
	data = df.copy()
	# data = df.set_index(["stock_id", "date_id", "seconds_in_bucket"])

	data["imbalance_buy_sell"] = data.imbalance_size * data.imbalance_buy_sell_flag

	params = ["imbalance_buy_sell", "reference_price", "matched_size", "wap_logreturns", "matched_ratio"]


	# data["wap_logreturns"] = np.log(data.wap/data.wap.shift(1))

	data["prev_wap"] = data.groupby(["date_id", "stock_id"]).wap.shift(1)

	# data["prev_wap"] = data["prev_wap"].fillna(0)

	data["wap_logreturns"] = np.log(data.wap / data.prev_wap)

	data["matched_ratio"] = data.imbalance_buy_sell_flag*data.imbalance_size/data.matched_size

	# data["matched_ratio"] = data["matched_ratio"].fillna(0)

	# data["wap_logreturns"] = data["wap_logreturns"].fillna(0)

	# data.replace([np.inf, -np.inf], np.nan, inplace=True)
	data['ewa6'] = data.groupby(["stock_id","date_id"]).wap.transform(lambda x : x.ewm(span=6, adjust=False).mean()).values
	# data['ema12'] = data.groupby(["stock_id","date_id"]).wap.ewm(span=12, adjust=False).mean().values
	data['ewa12'] = data.groupby(["stock_id","date_id"]).wap.transform(lambda x : x.ewm(span=12, adjust=False).mean()).values

	if full_data == True:
		return data

	data = data.fillna(0)
	features = ["wap_logreturns", "matched_ratio", "ewa6", "ewa12"]
	if train == 0:
		X = data[features].to_numpy()
		y = data.target.to_numpy()
		# y = np.array(data.target).reshape(-1,1)
		return X[np.isfinite(X).all(1)], y[np.isfinite(X).all(1)]
		# return X, y

	X = data[features].copy()

	# X.wap_logreturns = X.wap_logreturns.fillna(0)
	# X.matched_ratio = X.matched_ratio.fillna(0)
	# X.fillna(0,inplace=True)



	return X.to_numpy()


In [8]:
# df = make_data(train, full_data=True)

In [9]:
# test = train[(train.stock_id == 2) & (train.date_id == 3)].copy()


# test['ema12'] = test.groupby(["stock_id","date_id"]).wap.ewm(span=12, adjust=False).mean().values


In [10]:
# test.head()

In [11]:
# df[(df.stock_id == 2) & (df.date_id == 3)].head()

In [12]:


X, y = make_data(train)

X_train, X_test, y_train, y_test = train_test_split(X,y ,
                                   random_state=104,
                                   test_size=0.25,
                                   shuffle=True)

In [13]:
X

array([[ 0.00000000e+00,  2.37708291e-01,  1.00000000e+00,
         1.00000000e+00],
       [ 0.00000000e+00, -1.01450776e-01,  1.00000000e+00,
         1.00000000e+00],
       [ 0.00000000e+00, -1.66475317e-01,  1.00000000e+00,
         1.00000000e+00],
       ...,
       [-1.89779711e-04,  0.00000000e+00,  9.96034445e-01,
         9.96176198e-01],
       [-2.00196593e-05,  1.05609823e-02,  9.99176157e-01,
         9.99257476e-01],
       [ 2.80401765e-04, -7.82716195e-02,  1.00222807e+00,
         1.00214905e+00]])

In [14]:
model = LinearRegression()
model.fit(X_train,y_train)


LinearRegression()

In [15]:
model.coef_

predict_target = model.predict(X)

In [16]:
# train[train.seconds_in_bucket > 10].head()
# train.loc[train.stock_id==1,['target', 'time_id', 'stock_id', 'seconds_in_bucket']]
train[train.stock_id==1]['seconds_in_bucket'].unique()

array([  0,  10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120,
       130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250,
       260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380,
       390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510,
       520, 530, 540])

In [17]:
pd.DataFrame(predict_target).describe()

,0
count,5.237980e+06
mean,-4.660740e-02
std,6.459255e-01
min,-3.823449e+01
25%,-3.129765e-01
50%,-4.497998e-02
75%,2.256774e-01
max,4.946596e+01


In [18]:

stats.describe(predict_target)



DescribeResult(nobs=5237980, minmax=(-38.23448621164104, 49.465958443325604), mean=-0.04660739975342495, variance=0.41721974549408963, skewness=-0.11568956250430583, kurtosis=26.86687211182766)

In [19]:
import optiver2023

env = optiver2023.make_env()
iter_test = env.iter_test()


In [20]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    X = make_data(test, train=1)
    sample_prediction['target'] = model.predict(X)
    # sample_prediction['target'] = 0.5
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


In [21]:
!tail submission.csv
# sub = pd.read_csv("./example_test_files/sample_submission.csv")
sub = pd.read_csv("submission.csv")

test = pd.read_csv("./example_test_files/test.csv")

revealed = pd.read_csv("./example_test_files/revealed_targets.csv")


# test.merge

480_540_190,-0.04613770904363079
480_540_191,-0.016755107016805937
480_540_192,-0.049039698062453674
480_540_193,-0.03821257862728178
480_540_194,-0.06705947441063476
480_540_195,-0.06524115061829594
480_540_196,-0.058094121901455686
480_540_197,-0.02782702553138705
480_540_198,-0.04039747183183717
480_540_199,-0.07254233728250004


In [22]:
sub = sub[sub.target != "target"]
sub.target = sub.target.astype(float)

In [23]:
ans = train[["row_id", "target"]].merge(sub, on="row_id", how="right")

In [24]:
mae = np.mean(np.abs(ans.target_x - ans.target_y))
mae

5.279746546195513

5.27941